# Uniformar los datos satelitales de CHIRPS - diarios por estación

In [1]:
from netCDF4 import Dataset
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import time

In [2]:
from warnings import simplefilter
simplefilter(action="ignore", category=RuntimeWarning)
simplefilter(action="ignore", category=FutureWarning)

In [3]:
'''Definir periodo de análisis: fecha de inicio y fin. 
   Comprendido entre [1981-01-01, 2021-11-30]'''
fecha_inicio = '1981-01-01'
fecha_fin = '2020-12-31'

'''devuelve los datos de una variable'''
def variable_dato(data, var):
    v = data.variables[var][:]
    return v

def creardiario(cad):
    anio = cad[2].split('.')
    md_i = '-01-01'
    if anio[0] == '2021':
        md_f = '-11-30'
    else:
        md_f = '-12-31'
    d1 = anio[0]+md_i
    d2 = anio[0]+md_f
    data = creardf(d1, d2, 'D')
    return data

'''crear un DF vacío en un rango de fechas'''
def creardf(f_ini = '1981-01-01', f_fin = '2021-11-30', fr = "MS"):
    dat = pd.DataFrame(index = rango_fecha(f_ini, f_fin, fr))
    return dat

'''devuelve un rango de fechas mensual (default: <1981/01/01 - 2021/11/01, mensual>)'''
def rango_fecha(f_ini = '1981-01-01', f_fin = '2021-11-30', f = 'MS'):
    f_rango = pd.date_range(start=f_ini, end=f_fin, freq=f)
    return f_rango.date

'''guarda un archivo <data> con un nombre <nom>'''
def guardar(data, nom, path='../datos/1.0.variables/'):
    pth = path + nom
    data.to_csv(pth, index = False)

## Datos diarios a partir de archivos anuales con mediciones diarias

In [4]:
'''Leer archivo con las estaciones y coordenadas de referencia
   y el archivo con los nombres de los archivos diarios'''
est_cols = ['id','lon','lat','altitud','estacion']
dflista = pd.read_csv(r'../datos/1.0.Estaciones/estaciones_senamhi_generado.csv', usecols=est_cols)[est_cols]
dfanio = pd.read_csv(r'../datos/1.2.formato4_chirps/datos_anio.csv', header =None)

In [5]:
pp_diario = pd.DataFrame()

for anio in range(dfanio.shape[0]):
    nom_col = []
    diario = creardiario(dfanio.loc[anio,0].split('_'))
    dfpp = diario
    diario = diario.reindex(columns = ['precipitacion','lon_o','lat_o','lon_n','lat_n','estacion']) 
    p = r'../datos/1.2.formato4_chirps/' + dfanio.loc[anio,0]
    data = Dataset(p)
    lat = variable_dato(data,'latitude')
    lon = variable_dato(data,'longitude')
    time = variable_dato(data,'time')
    pp = variable_dato(data,'precip')
    for i in range(dflista.shape[0]):
        sq_dif_lat = (lat - dflista.iloc[i,2])**2
        sq_dif_lon = (lon - dflista.iloc[i,1])**2
        min_index_lat=sq_dif_lat.argmin()
        min_index_lon=sq_dif_lon.argmin()
        dt = np.arange(0,data.variables['time'].size)  
        for time_index in dt:
            diario.iloc[time_index,1] = dflista.iloc[i,1]
            diario.iloc[time_index,2] = dflista.iloc[i,2]
            diario.iloc[time_index,0] = (pp[time_index,min_index_lat,min_index_lon])
            diario.iloc[time_index,3] = lon[min_index_lon]
            diario.iloc[time_index,4] = lat[min_index_lat]
            diario.iloc[time_index,5] = dflista.iloc[i,4]
        nombre = '../datos/1.0.formato_csv_corregido/chirps_'+dflista.iloc[i,0].astype(str)+'_'+dflista.iloc[i,4]+'.csv'
        diario['precipitacion']=diario['precipitacion'].astype(float)
        diario['precipitacion']=diario['precipitacion'].round(3)
        if anio == 0:
            diario.to_csv(nombre,header=False )
        else:
            station = pd.read_csv(nombre, header=None, names=['precipitacion','lon_o','lat_o','lon_n','lat_n','estacion'])
            station = pd.concat ([station, diario])
            station.to_csv(nombre,header=False )  # Grabar Diarios por etacion
        dfpp = pd.concat([dfpp, diario['precipitacion']], axis=1)
        nom_col.append(str(dflista.iloc[i,0]))
    pp_diario = pd.concat([pp_diario, dfpp], axis=0)

pp_diario.columns = nom_col
pp_diario = pp_diario.reset_index()
pp_diario.rename(columns={'index':'fecha'},inplace=True)
pp_diario['fecha'] = pd.to_datetime(pp_diario['fecha'])
pp_diario = pp_diario[(pp_diario['fecha'] >= fecha_inicio) & (pp_diario['fecha'] <= fecha_fin)]
guardar(pp_diario, 'chirps_diario_pp.csv')

for i in range(dflista.shape[0]):
    nombre = '../datos/1.0.formato_csv_corregido/chirps_'+dflista.iloc[i,0].astype(str)+'_'+dflista.iloc[i,4]+'.csv'
    estacion = pd.read_csv(nombre, header=None, usecols = [i for i in range(2)])
    estacion.columns = ['fecha', 'pp']
    estacion['fecha'] = pd.to_datetime(estacion['fecha'])
    estacion = estacion[(estacion['fecha'] >= fecha_inicio) & (estacion['fecha'] <= fecha_fin)]
    estacion.to_csv(nombre, index=False)
    

In [6]:
'''Datos del dataframe generado'''
print(estacion.head(5))

       fecha     pp
0 1981-01-01  0.000
1 1981-01-02  3.296
2 1981-01-03  5.145
3 1981-01-04  0.000
4 1981-01-05  9.486
